# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 71 new papers today
          29 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/28 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.03608


extracting tarball to tmp_2301.03608...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.03609


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.03608/rnaas.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2301.04081...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.04087


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.04081/ms_V838Cyg.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'appendix' from 'tmp_2301.04081/appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2301.04087...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03689-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03689) | **Improving Star Cluster Age Estimates in PHANGS-HST Galaxies and the  Impact on Cluster Demographics in NGC 628**  |
|| Bradley C. Whitmore, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>James Lilly</mark>, <mark>Kiana Henny</mark>, <mark>Frank Bigiel</mark>, <mark>Stephen Hannon</mark>, <mark>Kathryn Kreckel</mark>, <mark>Francesca Pinna</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *27 pages, 17 figures (NOTE: 1, 5, 9, 12, 14 are lower resolution than in the journal)*|
|**Abstract**| A long-standing problem when deriving the physical properties of stellar populations is the degeneracy between age, reddening, and metallicity. When a single metallicity is used for all star clusters in a galaxy, this degeneracy can result in $`$catastrophic$'$ errors for old globular clusters. Typically, approximately 10 - 20 % of all clusters detected in spiral galaxies can have ages that are incorrect by a factor of ten or more. In this paper we present a pilot study for four galaxies (NGC 628, NGC 1433, NGC 1365, and NGC 3351) from the PHANGS-HST survey. We describe methods to correct the age-dating for old globular clusters, by first identifying candidates using their colors, and then reassigning ages and reddening based on a lower metallicity solution. We find that young $`$interlopers$'$ can be identified from their Halpha flux. CO (2-1) intensity or the presence of dust can also be used, but our tests show that they do not work as well. Improvements in the success fraction are possible at the $\sim$ 15 % level (reducing the fraction of catastrophic age-estimates from between 13 - 21 % to 3 - 8 %). A large fraction of the incorrectly age-dated globular clusters are systematically given ages around 100 Myr, polluting the younger populations as well. Incorrectly age-dated globular clusters significantly impact the observed cluster age distribution in NGC 628, which affects the physical interpretation of cluster disruption in this galaxy. For NGC 1365, we also demonstrate how to fix a second major age-dating problem, where very dusty young clusters with E(B-V) $>$ 1.5 mag are assigned old, globular-cluster like ages. Finally, we note the discovery of a dense population of $\sim$ 300 Myr clusters around the central region of NGC 1365. and discuss how this results naturally from the dynamics in a barred galaxy. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03811-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03811) | **The PHANGS-MUSE Nebula Catalogue**  |
|| B. Groves, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *27 pages, accepted by MNRAS*|
|**Abstract**| Ionized nebulae provide critical insights into the conditions of the interstellar medium (ISM). Their bright emission lines enable the measurement of physical properties, such as the gas-phase metallicity, across galaxy disks and in distant galaxies. The PHANGS--MUSE survey has produced optical spectroscopic coverage of the central star-forming discs of 19 nearby main-sequence galaxies. Here, we use the H{\alpha} morphology from this data to identify 30,790 distinct nebulae, finding thousands of nebulae per galaxy. For each nebula, we extract emission line fluxes and, using diagnostic line ratios, identify the dominant excitation mechanism. A total of 23,244 nebulae (75%) are classified as HII regions. The dust attenuation of every nebulae is characterised via the Balmer decrement and we use existing environmental masks to identify their large scale galactic environment (centre, bar, arm, interarm and disc). Using strong-line prescriptions, we measure the gas-phase oxygen abundances (metallicity) and ionization parameter for all HII regions. With this new catalogue, we measure the radial metallicity gradients and explore second order metallicity variations within each galaxy. By quantifying the global scatter in metallicity per galaxy, we find a weak negative correlation with global star formation rate and stronger negative correlation with global gas velocity dispersion (in both ionized and molecular gas). With this paper we release the full catalogue of strong line fluxes and derived properties, providing a rich database for a broad variety of ISM studies. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.03608-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.03608) | **The NANOGrav 12.5-year Data Set: Bayesian Limits on Gravitational Waves  from Individual Supermassive Black Hole Binaries**  |
|| Zaven Arzoumanian, et al. -- incl., <mark>Peter A. Gentile</mark>, <mark>Joseph Glaser</mark>, <mark>T. Joseph W. Lazio</mark>, <mark>Tingting Liu</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *20 pages, 12 figures. Submitted to ApJ*|
|**Abstract**| Pulsar timing array collaborations, such as the North American Nanohertz Observatory for Gravitational Waves (NANOGrav), are seeking nanohertz gravitational waves emitted by supermassive black hole binaries formed in the aftermath of galaxy mergers. We have searched for continuous waves from individual circular supermassive black hole binaries using the NANOGrav's recent 12.5-year data set. We created new methods to accurately model the uncertainties on pulsar distances in our analysis, and we implemented new techniques to account for a common red noise process in pulsar timing array data sets while searching for deterministic gravitational wave signals, including continuous waves. As we found no evidence for continuous waves in our data, we placed 95\% upper limits on the strain amplitude of continuous waves emitted by these sources. At our most sensitive frequency of 7.65 nanohertz, we placed a sky-averaged limit of $h_0 < $ $(6.82 \pm 0.35) \times 10^{-15}$, and $h_0 <$ $(2.66 \pm 0.15) \times 10^{-15}$ in our most sensitive sky location. Finally, we placed a multi-messenger limit of $\mathcal{M} <$ $(1.41 \pm 0.02) \times 10^9 M_\odot$ on the chirp mass of the supermassive black hole binary candidate 3C~66B. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04002-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04002) | **X-ray Polarimetry for Black Hole Spin Measurements]{X-ray Polarimetry as  a Tool to Measure the Black Hole Spin in Microquasars: Simulations of IXPE  Capabilities**  |
|| Romana Mikusincova, et al. -- incl., <mark>Wenda Zhang</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *11 pages, 10 figures*|
|**Abstract**| Measurements of the angular momentum (spin) of astrophysical black holes are extremely important, as they provide information on the black hole formation and evolution. We present simulated observations of a X-ray binary system with the Imaging X-ray Polarimetry Explorer (IXPE), with the aim to study the robustness of black hole spin and geometry measurements using X-ray polarimetry. As a representative example, we used the parameters of GRS 1915+105 in its former unobscured, soft state. In order to simulate the polarization properties, we modeled the source emission with a multicolor blackbody accounting for thermal radiation from the accretion disk, including returning radiation. Our analysis shows that the polarimetric observations in the X-ray waveband will be able to estimate both spin and inclination of the system with a good precision (without returning radiation we obtained for the lowest spin $\Delta a \leq 0.4$ (0.4/0.998 $\sim$ 40 \%) for spin and $\Delta i \leq 30^\circ$ (30$^\circ$/70$^\circ$ $\sim$ 43\%) for inclination, while for the higher spin values we obtained $\Delta a \leq 0.12$ ($\sim$ 12 \%) for spin and $\Delta i \leq 20^\circ$ ($ \sim $ 29\%) for inclination, within 1$ \sigma $ errors). When focusing on the case of returning radiation and treating inclination as a known parameter, we were able to successfully reconstruct spin and disk albedo in $\Delta a \leq 0.15$ ($\sim$ 15 \%) interval and $\Delta$ albedo $\leq 0.45$ (45 \%) intervals within 1$ \sigma $ errors. We conclude that X-ray polarimetry will be a useful tool to constrain black hole spins, in addition to timing and spectral-fitting methods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04055-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04055) | **PSR~J1910$-$5959A: A rare gravitational laboratory for testing white  dwarf models**  |
|| A. Corongiu, et al. -- incl., <mark>M. Kramer</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *18 pages, 12 figures. Abstract shortened and rephrased because of arXiv characters limits. Accepted for publication on A&A on January 9th 2023*|
|**Abstract**| PSRJ1910-5959A (J1910A) is a binary millisecond pulsar in a 0.837 day circular orbit around a helium white dwarf (HeWD) companion. This pulsar is located 6.3 arcmin away from the centre of the globular cluster NGC6752. Given the large offset, the association of the pulsar to NGC6752 has been debated. We have made use of two decades of archival Parkes 64-m "Murriyang" telescope data and recently carried out observations with the MeerKAT telescope. We obtained Pulse times of arrival using standard data reduction techniques and analysed using Bayesian pulsar timing techniques. We analysed the pulsar's total intensity and polarisation profile, to study the interstellar scattering along the line of sight, and the pulsar's geometry by applying the rotating vector model. We obtain precise measurements of several post-Keplerian parameters: the range $r=0.202(6)T_\odot$ and shape s=0.999823(4) of the Shapiro delay, from which we infer the orbital inclination to be $88.9^{+0.15}_{-0.14}\deg$ and the masses of both the pulsar and the companion to be $1.55(7)M_{\odot}$ and $0.202(6)M_{\odot}$ respectively; a secular change in the orbital period $\dot{P}_{\rm b}=-53^{+7.4}_{-6.0}\times 10^{-15}$\,s\,s$^{-1}$ that proves the association to NGGC6752 and a secular change in the projected semi-major axis of the pulsar $\dot{x}= -40.7^{+7.3}_{-8.2}\times10^{-16}$\,s\,s$^{-1}$ that is likely caused by the spin-orbit interaction from a misaligned HeWD spin, at odds with the likely isolated binary evolution of the system. We also discuss some theoretical models for the structure and evolution of WDs in NS-WD binaries by using J1910A's companion as a test bed. J1910A is a rare system for which several parameters of both the pulsar and the HeWD companion can be accurately measured. As such, it is a test bed to discriminate between alternative models for HeWD structure and cooling. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04056-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04056) | **Reverberation of pulsar wind nebulae (II): Anatomy of the "thin-shell''  evolution**  |
|| R. Bandiera, et al. -- incl., <mark>D. F. Torres</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *23 pages, 20 figures*|
|**Abstract**| During its early evolution, a pulsar wind nebula (PWN) sweeps the inner part of the supernova ejecta and forms a thin massive shell. Later on, when the shell has been reached by the reverse shock of the supernova remnant, the evolution becomes more complex, in most cases reverting the expansion into a compression: this later phase is called "reverberation". Computations done so far to understand this phase have been mostly performed in the thin-shell approximation, where the evolution of the PWN radius is assimilated to that of the swept-up shell under the effect of both the inner pressure from the PWN, and the outer pressure from the supernova remnant. Despite the thin-shell approach seems rather justifiable, its implementations have so far been inaccurate, and its correctness, never tested. The outer pressure was naively assumed to be scaled according to the Sedov solution (or a constant fraction of it) along the entire evolution. The thin-shell assumption itself fails along the process, being the shell no longer thin in comparison with the size of the PWN. Here, through a combination of numerical models, dimensional arguments, and analytic approximations, we present a detailed analysis of the interaction of the PWN with the supernova remnant. We provide a new analytic approximation of the outer pressure, beyond the Sedov solution, and a revised "thin-shell" able to reproduce results from numerical simulations. Finally, we compute the efficiency by which the PWN is compressed during reverberation over a wide population of sources. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04081-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04081) | **Two Different Weak Modulations in ab-type RR Lyrae Variable V838 Cyg,  and Potential Influence of Metal Abundance on Blazhko Modulation**  |
|| <mark>L. -J. Li</mark>, S. -B. Qian, X. -D. Shi, L. -Y. Zhu |
|*Appeared on*| *2023-01-11*|
|*Comments*| *18 pages, 13 figures*|
|**Abstract**| Noting the weakest modulation and relatively high metal abundance of the ab type RR Lyrae star V838 Cyg, we collected the photometric data of this star in several sky surveys to carry out an in-depth analysis. The O-C diagram shows that the pulsation period of V838 Cyg increases linearly on the large time scale. In the reanalysis of the high-precision Kepler data, we confirmed the modulation with the period of 59.45\pm0.04 days found earlier, and also found an additional weak modulation with a longer period (840\pm21 days). After a series of analyses, we incline to the view that the mechanisms leading the two modulations are different: the former is more similar to the typical Blazhko effect, while the mechanism leading to the latter may be an extrinsic factor. We also collected and compared the modulation and physical parameters of other Blazhko RR Lyrae stars from several works of literature, and find that there is a potential negative correlation between the modulation amplitude (or upper limit of amplitude) and the metal abundance. We infer that the relative metal-rich will promote the convection in the outer stellar atmosphere, and then inhibit those factors (turbulence, shock wave, etc.) that may cause Blazhko modulation. Future observations and research work can be carried out with reference to this viewpoint. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04087-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04087) | **The Low-Redshift Lyman Continuum Survey: Optically Thin and Thick Mg II  Lines as Probes of Lyman Continuum Escape**  |
|| Xinfeng Xu, et al. -- incl., <mark>Alaina Henry</mark>, <mark>Timothy Heckman</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *12 pages, 5 figures, 2 tables, accepted to ApJ*|
|**Abstract**| The Mg II 2796, 2803 doublet has been suggested to be a useful indirect indicator for the escape of Ly-alpha and Lyman continuum (LyC) photons in local star-forming galaxies. However, studies to date have focused on small samples of galaxies with strong Mg II or strong LyC emission. Here we present the first study of Mg II probing a large dynamic range of galaxy properties, using newly obtained high signal-to-noise, moderate-resolution spectra of Mg II for a sample of 34 galaxies selected from the Low-redshift Lyman Continuum Survey. We show that the galaxies in our sample have Mg II profiles ranging from strong emission to P-Cygni profiles, and to pure absorption. We find there is a significant trend (with a possibility of spurious correlations of ~ 2%) that galaxies detected as strong LyC Emitters (LCEs) also show larger equivalent widths of Mg II emission, and non-LCEs tend to show evidence of more scattering and absorption features in Mg II We then find Mg II strongly correlates with Ly-alpha in both equivalent width and escape fraction, regardless of whether the emission or absorption dominates the Mg II profiles. Furthermore, we present that, for galaxies categorized as Mg II emitters (MgE), one can adopt the information of Mg II, metallicity, and dust to estimate the escape fraction of LyC within a factor of 3. These findings confirm that Mg II lines can be used as a tool to select galaxies as LCEs and to serve as an indirect indicator for the escape of Ly-alpha and LyC. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.03689.md
    + _build/html/tmp_2301.03689/./sed_2_halpha_vs_co_jun_11_2022.png
    + _build/html/tmp_2301.03689/./7plot_ngc1365.png
    + _build/html/tmp_2301.03689/./sinan_2plot_dec_6_2022.png
exported in  _build/html/2301.03811.md
    + _build/html/tmp_2301.03811/./atlas_full/IC5332_muse_nebulae.png
    + _build/html/tmp_2301.03811/./figures/Figure_global_metal.png
    + _build/html/tmp_2301.03811/./figures/IC5332_38_spectralfit.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$
$\newcommand{\ngal}{ 17 }$
$\newcommand{\lea}{\mathrel{<\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand{\gea}{\mathrel{>\kern-1.0em\lower0.9ex\hbox{\sim}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}{ km s^{-1}}$
$\newcommand{$\bibtex$}{$\textsc${Bib}\!\TeX}$
$\newcommand{$\ngal$}{ 17 }$
$\newcommand{$\lea$}{\mathrel{<\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand{$\gea$}{\mathrel{>\kern-1.0em\lower0.9ex\hbox{\sim}}}$</div>



<div id="title">

# Improving Star Cluster Age Estimates in PHANGS-HST Galaxies andthe Impact on Cluster Demographics in NGC 628

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.03689-b31b1b.svg)](https://arxiv.org/abs/2301.03689)<mark>Appeared on: 2023-01-11</mark> - _27 pages, 17 figures (NOTE: 1, 5, 9, 12, 14 are lower resolution than in the journal)_

</div>
<div id="authors">

Bradley C. Whitmore, et al. -- incl., <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>James Lilly</mark></mark>, <mark><mark>Kiana Henny</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Stephen Hannon</mark></mark>, <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>Francesca Pinna</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** A long-standing problem when deriving the physical properties of stellar populationsis the degeneracy between age, reddening, and metallicity.When a single metallicity is usedfor all star clusters in a galaxy, this degeneracy can result in "catastrophic" errors for old globular clusters.Typically, approximately 10 -- 20\%of all clusters detected in spiral galaxies can have ages that are incorrect by a factor of ten or more.In this paper we presenta pilot study for four galaxies (NGC 628, NGC 1433, NGC 1365, and NGC 3351) from the PHANGS-HST survey. We describemethods to correct the age-dating for old globular clusters, by first identifying candidatesusing their colors, and then reassigning ages and reddening  based on a lower metallicity  solution.We find that young `Interlopers' can be identified from their  H$\alpha$flux. CO (2-1) intensity or the presence of dust can also be used, but our tests show that they do not work as well. Improvements in the success fraction are possible at the$\approx15$\%level (reducing the fraction of catastrophic age-estimates from  between 13 - 21\%to 3 - 8\%).A large fraction of the incorrectly age-dated globular clusters are systematically given ages around 100 Myr, polluting the  younger populations as well.Incorrectly age-dated globular clusters significantly impact the observed cluster age distribution in NGC 628, which affects the physical interpretation of cluster disruption in this galaxy.For NGC 1365, we also demonstrate how to fix a second major age-dating problem,where very dusty young clusters with$E(B-V)>1.5$mag are assigned old, globular-cluster like ages.Finally, we note the discovery of a dense population of$\approx300$Myr clusters around the central region of NGC 1365.and discuss how this results naturally from the dynamics in a barred galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2301.03689/./sed_2_halpha_vs_co_jun_11_2022.png" alt="Fig14" width="100%"/>

**Figure 14. -** 
Measured   H$\alpha$ flux vs. CO intensity for the clusters in our four sample galaxies. In each panel, the green line shows where objects appear to be associated with  the faintest HII regions and the yellow lines shows the corresponding line for the CO flux of the same HII regions. These lines provide estimates of where the line should be set to determine whether an objects is an interloper, i.e., a young objects with enough dust that it has colors appropriate for an old globular cluster. Snapshots of various objects are included to illustrate various points in the text. The three snapshots are Hubble images using B-V-I  (top), B-V-H$\alpha$(middle) and B-V-CO (bottom) images. The units for  H$\alpha$ flux are  10$^{-18}$ erg/s/cm$^2$/pixel and for CO intensity are K km s$^{-1}$. (*fig:co_halpha_4plot*)

</div>
<div id="div_fig2">

<img src="tmp_2301.03689/./7plot_ngc1365.png" alt="Fig12" width="100%"/>

**Figure 12. -** Similar to Figure \ref{fig:7plot-n1433} for NGC 1365. The primary difference is that the hybrid OGC/high-red solution is shown rather than the OGC-box and VI-limit solutions. In addition, the OGC-box is smaller, and green points are used to show  the high-red clusters. See text for details.  (*fig:7plot-n1365*)

</div>
<div id="div_fig3">

<img src="tmp_2301.03689/./sinan_2plot_dec_6_2022.png" alt="Fig6" width="100%"/>

**Figure 6. -** Illustration of how the age/reddening/metallicity degeneracy (left panel) can lead to the incorrect age-dating of old globular clusters, resulting in positions above a diagonal line in the log Age - $E(B-V)$ diagram (right panel). Using figures from \citet{deger22} for 17 PHANGS galaxies as the base, the figure  shows how the mismatch between the location of the observations of old globular clusters (which have low metallicity)  and the pink SED track used for the prediction (solar metallicity) results in three different regions of poor fits, (i.e., $\approx$ 3 Myr shown by the blue arrow and oval, $\approx$ 100 Myr shown by the green arrow and oval, and $\approx$ 1 Gyr shown by the  red arrow and  oval), rather than the correct solution at the end of the SED track at around 10 Gyr (i.e., log Age = 4).   (*fig:degen*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\change}[1]{{\color{red}#1}}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\nii}{[N \textsc{ii}]}$
$\newcommand{\sii}{[S \textsc{ii}]}$
$\newcommand{\oi}{[O \textsc{i}]}$
$\newcommand{\niion}{[N \textsc{i}]}$
$\newcommand{\hei}{[He \textsc{i}]}$
$\newcommand{\siii}{[S \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\Mgb}{Mg\textit{b}}$
$\newcommand{\NaI}{Na \textsc{I}}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\ha}{\mbox{\rmn{H}\alpha}}$
$\newcommand{\hb}{\mbox{\rmn{H}\beta}}$
$\newcommand{\Rtwofive}{\mbox{R_25}}$
$\newcommand{\kms}{\mathrm{km s}^{-1}}$
$\newcommand{\re}{R_\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\Lsun}{\mathrm{L}_{\sun}}$
$\newcommand{\reff}{r_{\rm eff}}$
$\newcommand{\DAP}{{\tt DAP}}$
$\newcommand{\MAPS}{{\tt MAPS}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\change$}[1]{{\color{red}#1}}$
$\newcommand{$\oiii$}{[O $\textsc${iii}]}$
$\newcommand{$\nii$}{[N $\textsc${ii}]}$
$\newcommand{$\sii$}{[S $\textsc${ii}]}$
$\newcommand{$\oi$}{[O $\textsc${i}]}$
$\newcommand{$\nii$on}{[N $\textsc${i}]}$
$\newcommand{$\hei$}{[He $\textsc${i}]}$
$\newcommand{$\sii$i}{[S $\textsc${iii}]}$
$\newcommand{$\oi$i}{[O $\textsc${ii}]}$
$\newcommand{$\Mgb$}{Mg\textit{b}}$
$\newcommand{$\NaI$}{Na $\textsc${I}}$
$\newcommand{$\hii$}{H $\textsc${ii}}$
$\newcommand{$\ha$}{\mbox{\rmn{H}\alpha}}$
$\newcommand{$\hb$}{\mbox{\rmn{H}\beta}}$
$\newcommand{$\Rtwofive$}{\mbox{R_25}}$
$\newcommand{$\kms$}{\mathrm{km s}^{-1}}$
$\newcommand{$\re$}{R_\mathrm{e}}$
$\newcommand{$\Msun$}{\mathrm{M}_{\sun}}$
$\newcommand{$\Lsun$}{\mathrm{L}_{\sun}}$
$\newcommand{$\re$ff}{r_{\rm eff}}$
$\newcommand{$\DAP$}{{\tt DAP}}$
$\newcommand{$\MAPS$}{{\tt MAPS}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The PHANGS--MUSE Nebular Catalogue

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.03811-b31b1b.svg)](https://arxiv.org/abs/2301.03811)<mark>Appeared on: 2023-01-11</mark> - _27 pages, accepted by MNRAS_

</div>
<div id="authors">

B. Groves, et al. -- incl., <mark><mark>K. Kreckel</mark></mark>, <mark><mark>E. Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** Ionized nebulae provide critical insights into the conditions of the interstellar medium (ISM). Their bright emission lines enable the measurement of physical properties, such as the gas-phase metallicity, across galaxy disks and in distant galaxies. The PHANGS--MUSE survey has produced optical spectroscopic coverage of the central star-forming discs of 19 nearby main-sequence galaxies. Here, we use the$\ha$morphology from this data to identify 30,790 distinct nebulae, finding thousands of nebulae per galaxy. For each nebula, we extract emission line fluxes and, using diagnostic line ratios, identify the dominant excitation mechanism. A total of23,244 nebulae (75\%) are classified as$\hii$regions. The dust attenuation of every nebulae is characterised via the Balmer decrement and we use existing environmental masks to identify their large scale galactic environment (centre, bar, arm, interarm and disc). Using strong-line prescriptions, we measure the gas-phase oxygen abundances (metallicity) and ionization parameter for all$\hii$regions.  With this new catalogue, we measure the radial metallicity gradients and explore second order metallicity variations within each galaxy. By quantifying the global scatter in metallicity per galaxy, we find a weak negative correlation with global star formation rate and stronger negative correlation with global gas velocity dispersion (in both ionized and molecular gas). With this paper we release the full catalogue of strong line fluxes and derived properties, providing a rich database for a broad variety of ISM studies.

</div>

<div id="div_fig1">

<img src="tmp_2301.03811/./atlas_full/IC5332_muse_nebulae.png" alt="Fig12" width="100%"/>

**Figure 12. -** The spatial distribution of nebulae in the galaxy IC5332. The background greyscale image shows the $\ha$ emission in log scale, and the colour of the nebulae indicates their intrinsic (dust corrected) $\ha$ luminosity. (*fig:atlas_IC5332*)

</div>
<div id="div_fig2">

<img src="tmp_2301.03811/./figures/Figure_global_metal.png" alt="Fig8" width="100%"/>

**Figure 8. -** Correlations between the global scatter in metallicity ($\sigma$(O/H)), after removing the first order radial gradient, as a function of galaxy integrated properties. In the top right of each plot we show the Pearson correlation coefficient of those quantities, $r$. The total stellar mass (M$_*$), total star formation rate (SFR), and gas fraction show weak or absent trends. More pronounced  correlations are seen with the ionized gas velocity dispersion, as measured across the full disc ($\sigma_{\rm H\alpha, disc}$) or just the $\hii$ regions ($\sigma_{\rm H\alpha, HII}$), as well as with the molecular gas velocity dispersion ($\sigma_{\rm CO}$).   (*fig:global_scatter*)

</div>
<div id="div_fig3">

<img src="tmp_2301.03811/./figures/IC5332_38_spectralfit.png" alt="Fig3" width="100%"/>

**Figure 3. -** A nebula spectrum from IC5332 (region ID 38, $L_{$\ha$}=10^{36.8}$ erg s$^{-1}$), showing a typical integrated spectrum (gold), including the underlying stellar (and weak nebula) continuum with clear strong emission and absorption lines (with the strong lines labelled) and visible residuals from the sky background subtraction. Overlaid is the resulting analysis pipeline fit from pPXF (blue dashed line), from which we extract the emission line fluxes and kinematics. The lower panel shows the relative residual of the spectral fit in indigo and relative spectral error in cyan ((Nebula-Fit)/Fit and error/fit, respectively). In the insert we show a zoom in of the $\ha$ and $\nii$ region, where the spectral lines can be seen more clearly. We also show the underlying continuum fit (dotted green line) revealing the underlying $\ha$$_{\rm abs}$ feature. (*fig:specfit*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

163  publications in the last 7 days.
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers